Hi,

I found the setup of pystan a little tricky. It needs a compiler. If you haven't set it up yet, I think you can find this link useful.

https://pystan2.readthedocs.io/en/latest/windows.html

In this tutorial, the difficulty is finding distutils.cfg. If your setting is default, I think you will find it in \anaconda3\Lib\distutils.


The following is our model and data that I have initially sorted out based on the professor's documents. According to the professor's prompt, I think we need the following four tasks：

    1. Compare the ability of your model to predict the winner of new (unseen) games to simple approaches, such as fraction of games won, number of games played, etc.

    2. Try evaluating how many games are required to accurately predict the players' skill levels / win probability by decreasing the amount of training data available and observing the performance. 
    
    3. Try evaluating how quickly you can determine a new players' skill by either random game choices or carefully chosen games (matched based on estimated skill level).  You can leave a player out of the inference process entirely, then slowly add their games in and see how quickly you are able to learn their relative position.
    
    4. Experiment with learning a more complex model, for example taking into account game features (player's selected character) or additional latent scores (such as offensive and defensive skill) along with a correspondingly more elaborate probability of win function.
    
    
 I believe we need find a way to evaluate our model. May be the prediction of loss?

In [64]:
import numpy as np
import pystan
import matplotlib.pyplot as plt
%matplotlib inline

In [65]:
skill_model = """
data {
  int<lower=1> N;             # Total number of players
  int<lower=1> E;             # number of games
  real<lower=0> scale;        # scale value for probability computation
  int<lower=0,upper=1> win[E]; # PA wins vs PB
  int PA[E];                  # player info between each game
  int PB[E];                  # 
}
parameters {
  vector [N] skill;           # skill values for each player
}

model{
  for (i in 1:N){ skill[i]~normal(0,3); }
  for (i in 1:E){
    win[i] ~ bernoulli_logit( (scale)*(skill[PA[i]]-skill[PB[i]]) );
  }   # win probability is a logit function of skill difference
}
"""

In [68]:
import pickle
try:     # load it if already compiled
    sm = pickle.load(open('skill_model.pkl', 'rb'))
except:  # ow, compile and save compiled model
    sm = pystan.StanModel(model_code = skill_model)
    with open('skill_model.pkl', 'wb') as f: pickle.dump(sm, f)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_c78eee36111c513d03cc709b3fedf086 NOW.


In [57]:
def load_data(path):
    with open(path, encoding='utf-8') as f: lines = f.read().split('\n')

    p = 0;playerid = {};
    for i in range(len(lines)):
        csv = lines[i].split(',')
        if len(csv) != 10: continue;   # parse error or blank line
        player0,player1 = csv[1],csv[4]
        if player0 not in playerid: playerid[player0]=p;p+=1
        if player1 not in playerid: playerid[player1]=p;p+=1

    nplayers = len(playerid)
    games = 0
    
    # Sparsifying parameters (discard some training examples):
    pKeep = 1.0   # fraction of edges to consider (immed. throw out 1-p edges)
    nEdge = 3     # try to keep nEdge opponents per player (may be more; asymmetric)
    nKeep = 5     # keep at most nKeep games per opponent pairs (play each other multiple times)

    nplays = np.zeros( (nplayers,nplayers) )
    PA, PB, win = [], [], []
    for i in range(len(lines)):
        csv = lines[i].split(',')
        if len(csv) != 10: continue;   # parse error or blank line
        a,b = playerid[csv[1]],playerid[csv[4]]
        aw,bw = csv[2]=='[winner]',csv[5]=='[winner]'
        if (np.random.rand() < pKeep):
            if (nplays[a,b] < nKeep) and ( ((nplays[a,:]>0).sum() < nEdge) or ((nplays[:,b]>0).sum() < nEdge) ):
                nplays[a,b] += 1; nplays[b,a]+=1; PA.append(a+1); PB.append(b+1);games+=1
                if aw:
                    win.append(1)
                else:
                    win.append(0)


    return nplayers, games, PA, PB, win

In [58]:
nplayers, games, PA, PB, win = load_data('train.csv')

In [59]:
print('summary: ')
print('# players', nplayers)
print('# games', games)
print('player A', PA[:10])
print('player B', PB[:10])
print('win', win[:10])

summary: 
# players 999
# games 4677
player A [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
player B [2, 3, 4, 5, 6, 7, 8, 9, 10, 9]
win [0, 0, 0, 0, 1, 1, 1, 0, 0, 1]


In [69]:
skill_data = {
    'N': nplayers,
    'E': games,
    'scale': 0.3,
    'win':win,
    'PA': PA,
    'PB': PB
}

In [70]:
fit = sm.sampling(data=skill_data, iter=1000, chains=4)

In [71]:
samples = fit.extract()

In [72]:
samples['skill'].shape

(2000, 999)

In [73]:
# Player 0 vs Player 1 prediction:
def logit(z): return 1./(1.+np.exp(-z))

# Use our model's win probability function (logistic of scaled difference)
#  using the predicted skill difference for each sample:
prob = logit( skill_data['scale']*(samples['skill'][:,0]-samples['skill'][:,1]) ).mean()

print(prob)

0.4911748826634101
